# February DS/AL-ML + BIA Data Jam - US Consumer Behavior

## Introduction

The project is to collaboratively evaluate the claim using real U.S. macroeconomic data from Federal Reserve Economic Data (FRED) and present a clear, evidence-based conclusion. We will explore how inflation has structurally altered consumer spending, saving, and borrowing habits.

## 1. Environment Setup & Data Loading

In [26]:
# Import necessary libraries
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [27]:
# Load the starter datasets
credit_owned = pd.read_csv('../data/CREDIT_OWNED.csv')
personal_expenditure = pd.read_csv('../data/PERSONAL_EXPENDITURE.csv')
saving_rate = pd.read_csv('../data/SAVING_RATE.csv')
cpi = pd.read_csv('../data/cpiaucsl.csv')

## 2. Initial Data Inspection
Before merging, we must understand the shape, completeness, and historical timelines of our individual datasets.

In [28]:
# Display the first few rows of each dataset
print("Credit Owned Dataset:")
display(credit_owned.head())

print("\nPersonal Expenditure Dataset:")
display(personal_expenditure.head())

print("\nSaving Rate Dataset:")
display(saving_rate.head())

Credit Owned Dataset:


,observation_date,TOTALSL
0,1943-01-01,6577.83
1,1943-02-01,6463.04
2,1943-03-01,6234.21
3,1943-04-01,6125.75
4,1943-05-01,5936.26



Personal Expenditure Dataset:


,observation_date,PCEC96
0,2007-01-01,11181.0
1,2007-02-01,11178.2
2,2007-03-01,11190.7
3,2007-04-01,11201.5
4,2007-05-01,11218.0



Saving Rate Dataset:


,observation_date,PSAVERT
0,1959-01-01,11.3
1,1959-02-01,10.6
2,1959-03-01,10.3
3,1959-04-01,11.2
4,1959-05-01,10.6


In [29]:
# Determining the size of all the DataFrames

# Display the shape of each dataset
print("Shape of Credit Owned Dataset:", credit_owned.shape)
print("Shape of Personal Expenditure Dataset:", personal_expenditure.shape)
print("Shape of Saving Rate Dataset:", saving_rate.shape)
print("Shape of CPI Dataset:", cpi.shape)

Shape of Credit Owned Dataset: (995, 2)
Shape of Personal Expenditure Dataset: (227, 2)
Shape of Saving Rate Dataset: (803, 2)
Shape of CPI Dataset: (949, 2)


In [30]:
# Check for missing values and duplicates
for name, df in zip(['Credit', 'Expenditure', 'Saving', 'CPI'], [credit_owned, personal_expenditure, saving_rate, cpi]):
    print(f"{name} - Missing: {df.isnull().sum().sum()} | Duplicates: {df.duplicated().sum()}")

Credit - Missing: 0 | Duplicates: 0
Expenditure - Missing: 0 | Duplicates: 0
Saving - Missing: 0 | Duplicates: 0
CPI - Missing: 1 | Duplicates: 0


In [31]:
# Display informative summary of each dataset
print("Credit Owned Dataset Info:")
credit_owned.info()

print("\nPersonal Expenditure Dataset Info:")
personal_expenditure.info()

print("\nSaving Rate Dataset Info:")
saving_rate.info()

print("\nCPI Dataset Info:")
cpi.info()

Credit Owned Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 995 entries, 0 to 994
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  995 non-null    object 
 1   TOTALSL           995 non-null    float64
dtypes: float64(1), object(1)
memory usage: 15.7+ KB

Personal Expenditure Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  227 non-null    object 
 1   PCEC96            227 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.7+ KB

Saving Rate Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803 entries, 0 to 802
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  

In [32]:
# Display descriptive statistics of each dataset
print("Credit Owned Dataset Description:")
display(credit_owned.describe())

print("\nPersonal Expenditure Dataset Description:")
display(personal_expenditure.describe())

print("\nSaving Rate Dataset Description:")
display(saving_rate.describe())

print("\nCPI Dataset Description:")
display(cpi.describe())

Credit Owned Dataset Description:


,TOTALSL
count,9.950000e+02
mean,1.230814e+06
std,1.481781e+06
min,5.354360e+03
25%,7.464136e+04
50%,4.809945e+05
75%,2.215879e+06
max,5.084831e+06



Personal Expenditure Dataset Description:


,PCEC96
count,227.000000
mean,13176.984581
std,1724.263802
min,11068.000000
25%,11555.000000
50%,12884.000000
75%,14455.350000
max,16715.400000



Saving Rate Dataset Description:


,PSAVERT
count,803.000000
mean,8.404857
std,3.424809
min,1.400000
25%,5.700000
50%,8.300000
75%,11.100000
max,31.800000



CPI Dataset Description:


,CPIAUCSL
count,948.000000
mean,124.062082
std,89.409534
min,21.480000
25%,32.825000
50%,109.650000
75%,199.950000
max,326.588000


## 3. Clean & Merge Data
**Observation:** Our inspection reveals a significant misalignment in our historical data:
* `credit_owned` contains 995 rows (starting in 1943).
* `saving_rate` contains 803 rows (starting in 1959).
* `personal_expenditure` contains 227 rows (starting in 2007).

**Action:** Because machine learning models require uniform matrices without missing values across features, we cannot simply concatenate these files. We must standardize the date columns to a `datetime` object and perform an **inner join**. This will naturally trim our timeline to start around 2007 (the earliest shared date across all datasets), ensuring we are only analyzing periods where we have a complete macroeconomic picture.

In [33]:
# Standardize the date column names and cast to datetime objects
for df in [credit_owned, saving_rate, personal_expenditure, cpi]:
    # Rename the first column to 'DATE' regardless of what FRED named it
    df.rename(columns={df.columns[0]: 'DATE'}, inplace=True)
    df['DATE'] = pd.to_datetime(df['DATE'])

# Perform the Inner Merge on the unified DATE key
master_df = personal_expenditure.merge(saving_rate, on='DATE', how='inner') \
                                .merge(credit_owned, on='DATE', how='inner') \
                                .merge(cpi, on='DATE', how='inner')

# Rename columns to be more descriptive
master_df.rename(columns={
    'PCEC96': 'Expenditure_Billions',
    'PSAVERT': 'Saving_Rate_Pct',
    'TOTALSL': 'Credit_Owned_Billions',
    'CPIAUCSL': 'CPI_Index'
}, inplace=True)

# Ensure chronological order
master_df.sort_values('DATE', inplace=True)

## 4. Feature Engineering
Raw nominal dollars and static percentages are difficult for ML algorithms to interpret over long periods. We will engineer new features that capture *behavioral momentum* and *macroeconomic stress*.

In [34]:
# 1. Ratio Features
master_df['Credit_to_Spend_Ratio'] = master_df['Credit_Owned_Billions'] / master_df['Expenditure_Billions']

# 2. Year-over-Year (YoY) Growth Features
master_df['Spend_YoY_Growth'] = master_df['Expenditure_Billions'].pct_change(periods=12) * 100
master_df['Credit_YoY_Growth'] = master_df['Credit_Owned_Billions'].pct_change(periods=12) * 100
master_df['Inflation_YoY'] = master_df['CPI_Index'].pct_change(periods=12) * 100

# 3. Regime Categorization
def assign_regime(date):
    if date < pd.to_datetime('2020-03-01'):
        return '1_Pre_Covid'
    elif date < pd.to_datetime('2021-06-01'):
        return '2_Covid_Stimulus'
    else:
        return '3_Post_Inflation_Shock'

master_df['Regime'] = master_df['DATE'].apply(assign_regime)

# Drop the first 12 months which now contain NaNs due to the YoY calculation
master_df.dropna(inplace=True)

### Engineered Features Explained
To successfully model consumer behavior, we derived the following indicators:

* **`Credit_to_Spend_Ratio`:** A proxy for financial health. It measures how much outstanding debt consumers hold relative to their current spending levels. An increasing ratio suggests consumers are relying heavier on credit to fund their lifestyle.
* **`Spend_YoY_Growth` & `Credit_YoY_Growth`:** Year-over-year percentage changes. Using a 12-month lookback completely removes annual seasonality (e.g., the December holiday shopping spike) and allows our models to measure true behavioral momentum.
* **`Inflation_YoY`:** The core driver of our hypothesis. This translates the raw CPI index into the actual inflation rate experienced by consumers over the last 12 months. 
* **`Regime`:** A categorical flag that splits the timeline into three distinct economic eras (`1_Pre_Covid`, `2_Covid_Stimulus`, and `3_Post_Inflation_Shock`). This is critical for detecting structural breaks, as it allows our models to compare pre-shock behavior against post-shock behavior.

In [35]:
# Save the enriched dataset
master_df.to_csv('../data/master_df.csv', index=False)
print(f"Data successfully cleaned and saved! Final shape: {master_df.shape}")
display(master_df.head())

Data successfully cleaned and saved! Final shape: (214, 10)


,DATE,Expenditure_Billions,Saving_Rate_Pct,Credit_Owned_Billions,CPI_Index,Credit_to_Spend_Ratio,Spend_YoY_Growth,Credit_YoY_Growth,Inflation_YoY,Regime
12,2008-01-01,11333.2,2.6,2619427.65,212.174,231.128688,1.361238,6.569798,4.294696,1_Pre_Covid
13,2008-02-01,11293.9,3.0,2634496.42,212.687,233.267199,1.035050,6.657618,4.142959,1_Pre_Covid
14,2008-03-01,11322.1,2.9,2645603.64,213.448,233.667221,1.174189,6.487213,3.974904,1_Pre_Covid
15,2008-04-01,11340.5,2.4,2654243.23,213.942,234.049930,1.240905,6.436682,3.903761,1_Pre_Covid
16,2008-05-01,11361.6,6.8,2660193.15,215.208,234.138955,1.280086,5.983113,4.088414,1_Pre_Covid


## Data Analysis